In [70]:
from collections import Counter, defaultdict
from nltk.tokenize import word_tokenize
import pickle
import string
import nltk
from nltk.tokenize import word_tokenize
from searchEngine import MySearchEngine

In [85]:
class entityDatabase:
    def __init__(self, pickle_path):
        self.ent_dict = defaultdict(list)
        pfile = open(pickle_path, mode='rb')
        p = pickle.load(pfile)
        #pickle_path example: "C:\\Users\\User\\Desktop\\beaver\\NumberWon\\numberwon\\entity\\test.pickle"
        self.entize(p)
        #self.engine = MySearchEngine(file=pfile)
        
    def get_by_id(self, id):
        return self.ent_dict[id]
    
    def entize(self, pickle):
        for key, val in pickle.items():
            tokens = nltk.word_tokenize(val)
            pos = nltk.pos_tag(tokens)
            named_entities = nltk.ne_chunk(pos, binary = True)
            for i in range(0, len(named_entities)):
                ents = named_entities.pop()
                if getattr(ents, 'label', None) != None and ents.label() == "NE":
                    z = list(zip(*[ne for ne in ents]))[0]
                    z = " ".join(z)
                    self.ent_dict[key].append(z)

    #fix!!!             
    def searchNentity(self, qword):
        topdoc = engine.query(qword)[0][0]
        return top_entity_associated_with_item(qword,engine.raw_text[topdoc]) #Megan's method

    def docsearch(self, qword):
        topdoc = engine.query(qword)[0][0]
        raw = engine.raw_text[topdoc] #whole doc
        return re.match(r'(?:[^.:;]+[.:;]){1}', raw).group() #first sentence
    #fix!!!
    
    def top_entity_associated_with_item(self, item, weighted=True, rangee=5, most_c=10):
        #search for item.
            #for i in feed. if i == feed:
        #create a list of words that are close to word in proximity
        #score based on proximity to word.
        #documents is already a list
        word_freq = Counter()
        for i in self.ent_dict:
            for x in range(len(self.ent_dict[i])):
                if self.ent_dict[i][x] == item:
                    for z in range(rangee):
                        if not (x - z < 0):
                            if self.ent_dict[i][x-z] != self.ent_dict[i][x]:
                                if weighted:
                                    word_freq[self.ent_dict[i][x-z]] += 1/abs((x -(x-z)))
                                else:
                                    word_freq[self.ent_dict[i][x-z]] += 1
                        if not (x + z > (len(self.ent_dict[i]) - 1)):
                            if self.ent_dict[i][x+z] != self.ent_dict[i][x]:
                                if weighted:
                                    word_freq[self.ent_dict[i][x+z]] += 1/abs((x -(x+z)))
                                else:
                                    word_freq[self.ent_dict[i][x+z]] += 1
        return word_freq.most_common()
    
    def top_entity_dict(self, item, most_c=10):
        #documents is already a list
        #turn each list into a counter, add all counters together. 
        mega_counter = Counter()
        for i in self.ent_dict:
            #get list of counters etc
            if item in self.ent_dict[i]:
                c = Counter(self.ent_dict[i])
                del c[item]
                mega_counter += c
        return mega_counter.most_common(most_c)

In [86]:
edatb = entityDatabase("/Users/megankaye/Desktop/BeaverWorks/Work/NumberWon/numberwon/entity/testfeed.pickle")

In [87]:
edatb.ent_dict

defaultdict(list,
            {'http://feeds.reuters.com/~r/reuters/entertainment/~3/-Umx5yPXsw8/us-television-stephencolbert-idUSKBN1AC37K': ['Vladimir Putin',
              'Russian',
              'Trump',
              'Colbert',
              'CBS Corp',
              'Stephen Colbert',
              'Late Show',
              'Colbert',
              'Cartoon',
              'Trump',
              'Late Show',
              'CBS',
              'Colbert',
              'White House',
              'Trump',
              'Stephen Colbert',
              'Donald Trump',
              'LOS ANGELES',
              'Kauzlarich',
              'U.S.',
              'Ohio',
              'Cleveland',
              'Republican National Convention',
              'Stephen Colbert',
              'CBS',
              'Showtime',
              'Donald Trump',
              'Stephen Colbert'],
             'http://feeds.reuters.com/~r/reuters/entertainment/~3/2COukeUYzo0/us-film-wonderwoman-

In [88]:
edatb.top_entity_associated_with_item("Katy Perry", weighted=True, rangee=5)

[('MTV Video Music Awards', 2.0),
 ('Pop', 2.0),
 ('Britain', 0.5),
 ('FILE', 0.5),
 ('VMA', 0.5),
 ('Glastonbury Festival', 0.3333333333333333),
 ('Pyramid Stage', 0.3333333333333333),
 ('MTV', 0.3333333333333333),
 ('Somerset', 0.25),
 ('Worthy Farm', 0.25),
 ('Perry', 0.25)]

In [89]:
edatb.top_entity_dict("Katy Perry", most_c=10)

[('MTV Video Music Awards', 3),
 ('MTV', 2),
 ('VMA', 2),
 ('Pop', 2),
 ('Perry', 2),
 ('Rhythm', 1),
 ('American Idol', 1),
 ('Glastonbury Festival', 1),
 ('Taylor Swift', 1),
 ('Calvin Harris', 1)]